# Live projections of the steady state for the COVID19 pandemic

## Import the latest data
 * The world data comes from the Open-Covid-19 Github Projects
 * The US data comes from the https://www.covidtracking.com project
 * The Italian data comes from https://github.com/pcm-dpc/COVID-19, the official github account of the department of the Civil Protection Department of the Italian government.


In [0]:
from datetime import datetime, timedelta, timezone
import dateutil.parser as du_parser
import pandas as pd
import eloader as el

# configuration: countries to analyze
COUNTRIES = ['Italy', 'South Korea', 'China', 'United States of America', 'India', 'Brazil']
X_KEY='X'

# load from the data loader helper
(df_world_daily, _) = el.load_opencovid19_data()
(df_us_daily, _, _) = el.load_covidtracking_us_data()
(df_it_daily, _) = el.load_pcmdpc_it_data()

df_countries_daily = el.fuse_daily_sources(df_world_daily, df_us_daily, df_it_daily)
el.add_canonical_differentials(df_countries_daily)
df_countries_daily = el.cleanup_canonical(df_countries_daily)

df_logistics = df_countries_daily[['X', 'Confirmed', 'Recovered', 'Deaths', 'Hospitalized', 'CountryName']]

## Predicting the size of the infection
We use a logistic model and fit it to the data. Granted that the logistic model will differ from reality, but it's an easy and explainable model that can be used for now to model the slow spread of the virus to a population.

In [0]:
import numpy as np
from scipy.optimize import curve_fit, fsolve

def logistic_model(x, a, b, c):
    return c / (1 + np.exp(-(x - b) / a))


def logistic_model_modified(x, a, b, c, d, e):
    return c / (1 + np.exp(-(x - b) / a)) + d * (x - b) + e


# fit the data to the model (find the model variables that best approximate)
def predict_logistic_maximum(df, y_key):
    samples = df.shape[0]
    x_days = df[X_KEY].tolist()
    y_cases = df[y_key].tolist()
    initial_guess = 5, 90, 200000 # speed, peak, amplitude

    fit = curve_fit(logistic_model, x_days, y_cases, p0=initial_guess, maxfev=9999)

    # parse the result of the fit
    speed, x_peak, y_max = fit[0]
    speed_error, x_peak_error, y_max_error = [np.sqrt(fit[1][i][i]) for i in [0, 1, 2]]

    # find the "end date", as the x (day of year) where the function reaches 99.99%
    end = int(fsolve(lambda x: logistic_model(x, speed, x_peak, y_max) - y_max * 0.9999, x_peak))

    return x_days, y_cases, speed, x_peak, y_max, x_peak_error, y_max_error, end, samples


# print results
def print_prediction(df, y_key, label):
    x, y, speed, x_peak, y_max, x_peak_error, y_max_error, end, samples = predict_logistic_maximum(df, y_key)
    print(label + "'s prediction: " +
          "maximum cases: " + str(int(round(y_max))) +
          " (± " + str(int(round(y_max_error))) + ")" +
          ", peak at calendar day: " + str(int(round(x_peak))) +
          " (± " + str(round(x_peak_error, 2)) + ")" +
          ", ending on calendar day: " + str(end))
    return y_max


for country_name in COUNTRIES:
    df = df_logistics[df_logistics['CountryName'] == country_name]
    print(country_name + ':')
    print_prediction(df[:-2], 'Confirmed', "  2 days ago")
    print_prediction(df[:-1], 'Confirmed', "  yesterday")
    pred = print_prediction(df, 'Confirmed', "  today")
    print()

### Plot 1. data & projections, for today and the former 2 days

In [0]:
import matplotlib.pyplot as plt
plt.rc('font', size=14)


def add_real_data(df, y_key, label, color=None):
    x = df[X_KEY].tolist()
    y = df[y_key].tolist()
    plt.scatter(x, y, label="Data (" + label + ")", c=color)


def add_logistic_curve(df, y_key, label, **kwargs):
    x, _, speed, x_peak, y_max, _, _, end, _ = predict_logistic_maximum(df, y_key)
    x_range = list(range(int(min(x)), int(end)))
    plt.plot(x_range,
             [logistic_model(i, speed, x_peak, y_max) for i in x_range],
             label="Logistic model (" + label + "): " + str(int(round(y_max))),
             **kwargs)
    return y_max


def label_and_show_plot(plt, title, y_max=None):
    plt.title(title)
    plt.xlabel("Day of the year, 2020")
    # plt.ylabel("Total number of infected people")
    if (y_max):
        plt.ylim(0, y_max * 1.1)
    plt.legend()
    plt.show()


# Plot2
for country_name in COUNTRIES:
    df_country = df_logistics[df_logistics['CountryName'] == country_name]
    for metric_name in ['Confirmed', 'Deaths']:
        
        df = df_country[[X_KEY, metric_name]]
        df = df[df[metric_name].notna()]
        if df.empty: continue

        plt.rc('font', size=14)
        fig = plt.figure(figsize=(14, 10))
        fig.patch.set_facecolor('white')
        add_real_data(df[:-2], metric_name, "2 days ago")
        add_real_data(df[-2:-1], metric_name, "yesterday")
        add_real_data(df[-1:], metric_name, "today")
        add_logistic_curve(df[:-2], metric_name, "2 days ago", dashes=[8, 8])
        add_logistic_curve(df[:-1], metric_name, "yesterday", dashes=[4, 4])
        y_max = add_logistic_curve(df, metric_name, "today")
        label_and_show_plot(plt, country_name + " - " + metric_name + " forecast", y_max)